In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
! pip install transformers -q

     |████████████████████████████████| 573kB 1.4MB/s 
     |████████████████████████████████| 1.0MB 4.7MB/s 
     |████████████████████████████████| 870kB 8.4MB/s 
     |████████████████████████████████| 3.7MB 6.9MB/s 


In [0]:
import csv
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.metrics import *
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [0]:
import os
os.chdir('/gdrive/My Drive/DEFINITION EXTRACTION/Definition')

In [0]:
model_name="bert-base-cased"
#model_name="bert-base-uncased"
#model_name="roberta-base"
#model_name="xlnet-base-cased"

In [8]:
merged = pd.read_csv('WCL_cased_symbols.csv')
merged.head(2)

,label,Psentence,Length,Dep1,Dep2,DepLabel
0,1,A Pianist is a person who plays the piano.,9,Pianist is is is person person plays plays piano,A Pianist person . a plays who piano the,nsubj ROOT ROOT ROOT attr attr relcl relcl dobj
1,0,Pingala has a sunlike nature and male energy.,8,has has has nature nature nature nature energy,Pingala nature . a sunlike and energy male,ROOT ROOT ROOT dobj dobj dobj dobj conj


In [0]:
def seq_len(row):
  return len(row['Psentence'].split())

merged['Length'] = merged.apply(seq_len, axis=1)

In [10]:
merged = merged[merged['Length']>4]
merged.shape

(4680, 6)

## Transformer

In [11]:
 %tensorflow_version 1.x 
from transformers import BertTokenizer, GPT2Tokenizer, AutoTokenizer
from transformers import BertConfig, BertForSequenceClassification, AutoModel, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name, output_attentions=True)

TensorFlow 1.x selected.


## LM Training

In [23]:
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

t,v  = train_test_split(merged, test_size=0.3)

with open('deft.train.raw', 'w') as f:
  for i,row in tqdm(t.iterrows()):
    f.write(row['Psentence']+'\n')

with open('deft.test.raw', 'w') as f:
  for i,row in tqdm(v.iterrows()):
    f.write(row['Psentence']+'\n')


In [24]:
! rm -rf bert_ft
! mkdir bert_ft
! python train_lm.py \
    --output_dir=bert_ft \
    --model_type=bert \
    --model_name_or_path=bert-base-cased \
    --do_train \
    --train_data_file=./deft.train.raw \
    --do_eval \
    --eval_data_file=./deft.test.raw \
    --mlm\

04/08/2020 07:01:05 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/08/2020 07:01:05 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
04/08/2020 07:01:05 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1

## Preprocessing

In [14]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64
X = [tokenizer.encode(x, add_special_tokens=True) for x in merged['Psentence']]
X = pad_sequences(X, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Using TensorFlow backend.


In [0]:
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in X:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
attention_masks = np.array(attention_masks)

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [0]:
y = np.array(merged['label'])

# Training

In [18]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [0]:
from sklearn.utils.extmath import softmax
from sklearn.metrics import classification_report, f1_score


def flat_accuracy(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)


In [25]:
from sklearn.model_selection import KFold,StratifiedKFold
from tqdm import tqdm
from transformers import *

auto_model = None
f1_scores, p_scores, r_scores = [], [], []

kf = StratifiedKFold(10, shuffle=True, random_state=2018)
for f,(tr, val) in enumerate(kf.split(X, y)):
    print(f'--------------------------------------------------------------------------------Fold # {f}--------------------------------------------------------------------')

    X_train, X_val = X[tr], X[val]
    y_train, y_val = y[tr], y[val]
    X_train_masks, X_val_masks = attention_masks[tr], attention_masks[val]
                                         
    # Convert all of our data into torch tensors, the required datatype for our model
    X_train = torch.tensor(X_train)
    X_val = torch.tensor(X_val)

    y_train = torch.tensor(y_train)
    y_val = torch.tensor(y_val)

    X_train_masks = torch.tensor(X_train_masks)
    X_val_masks = torch.tensor(X_val_masks)

    batch_size = 32

    # Create an iterator of our data with torch DataLoader 
    train_data = TensorDataset(X_train, X_train_masks, y_train)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(X_val, X_val_masks, y_val)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


    ## Model
    del auto_model
    # auto_model = AutoModelForSequenceClassification.from_pretrained('./bert_ft', num_labels= len(np.unique(y_train)))
    auto_model = BertForSequenceClassification.from_pretrained('./bert_ft', output_attentions=False, num_labels= len(np.unique(y_train)))

    auto_model.to(device)

    # Optimizer
    num_total_steps = 1000
    num_warmup_steps = 100
    lr = 2e-6
    param_optimizer = list(auto_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    

    auto_model.train()  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    n_epochs = 4

    for epoch in (range(n_epochs)):  
      # Training Loop
      for step, batch in (enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()
        outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
          
      auto_model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # Validation Loop
      yt, yp = [], []
      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          outputs = auto_model(b_input_ids, 
                              token_type_ids=None,
                              attention_mask=b_input_mask,
                              labels= b_labels)        
          loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        
        preds = softmax(logits)
        pred_ids = np.argmax(preds, axis=1).flatten()

        label_ids = b_labels.to('cpu').numpy()

        eval_loss += loss.item()
        eval_accuracy += flat_accuracy(logits, label_ids)

        yt = yt + label_ids.tolist()
        yp = yp + pred_ids.tolist()

        nb_eval_steps +=1
        
      print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                      tr_loss/nb_tr_steps,
                                                                      eval_loss/nb_eval_steps, 
                                                                    ))
    #if epoch == n_epochs:
    z = classification_report(yt, yp, output_dict=True)
    f1_scores.append(z['1']['f1-score'])
    p_scores.append(z['1']['precision'])    
    r_scores.append(z['1']['recall'])
    
    print(classification_report(yt, yp))   


--------------------------------------------------------------------------------Fold # 0--------------------------------------------------------------------
Epoch 0 | Train loss: 0.43265664306553925 | Validation Loss: 0.2864905774593353 
Epoch 1 | Train loss: 0.3117167953627579 | Validation Loss: 0.14560329417387644 
Epoch 2 | Train loss: 0.23800540099277942 | Validation Loss: 0.11043802698453267 
Epoch 3 | Train loss: 0.1920161129568111 | Validation Loss: 0.10155531018972397 
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       280
           1       0.96      0.96      0.96       188

    accuracy                           0.97       468
   macro avg       0.97      0.97      0.97       468
weighted avg       0.97      0.97      0.97       468

--------------------------------------------------------------------------------Fold # 1--------------------------------------------------------------------
Epoch 0 | Train loss: 0.4136784520

In [26]:
np.mean(np.array(p_scores)), np.mean(np.array(r_scores)), np.mean(np.array(f1_scores))

(0.9624533024464617, 0.9727585618386619, 0.9673992321810259)

In [0]:
from sklearn.model_selection import KFold,StratifiedKFold
from tqdm import tqdm

auto_model = None
f1_scores, p_scores, r_scores = [], [], []

kf = StratifiedKFold(10, shuffle=True, random_state=2018)
for f,(tr, val) in enumerate(kf.split(X, y)):
    print(f'--------------------------------------------------------------------------------Fold # {f}--------------------------------------------------------------------')

    X_train, X_val = X[tr], X[val]
    y_train, y_val = y[tr], y[val]
    X_train_masks, X_val_masks = attention_masks[tr], attention_masks[val]
                                         
    # Convert all of our data into torch tensors, the required datatype for our model
    X_train = torch.tensor(X_train)
    X_val = torch.tensor(X_val)

    y_train = torch.tensor(y_train)
    y_val = torch.tensor(y_val)

    X_train_masks = torch.tensor(X_train_masks)
    X_val_masks = torch.tensor(X_val_masks)

    batch_size = 32

    # Create an iterator of our data with torch DataLoader 
    train_data = TensorDataset(X_train, X_train_masks, y_train)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(X_val, X_val_masks, y_val)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


    ## Model
    del auto_model
    auto_model = AutoModelForSequenceClassification.from_pretrained(model_name, output_attentions=True, num_labels= len(np.unique(y_train)))
    auto_model.to(device)

    # Optimizer
    num_total_steps = 1000
    num_warmup_steps = 100
    lr = 5e-6
    param_optimizer = list(auto_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    

    auto_model.train()  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    n_epochs = 4

    for epoch in (range(n_epochs)):  
      # Training Loop
      for step, batch in (enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()
        outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
          
      auto_model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # Validation Loop
      yt, yp = [], []
      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          outputs = auto_model(b_input_ids, 
                              token_type_ids=None,
                              attention_mask=b_input_mask,
                              labels= b_labels)        
          loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        
        preds = softmax(logits)
        pred_ids = np.argmax(preds, axis=1).flatten()

        label_ids = b_labels.to('cpu').numpy()

        eval_loss += loss.item()
        eval_accuracy += flat_accuracy(logits, label_ids)

        yt = yt + label_ids.tolist()
        yp = yp + pred_ids.tolist()

        nb_eval_steps +=1
        
      print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                      tr_loss/nb_tr_steps,
                                                                      eval_loss/nb_eval_steps, 
                                                                    ))
    #if epoch == n_epochs:
    z = classification_report(yt, yp, output_dict=True)
    f1_scores.append(z['1']['f1-score'])
    p_scores.append(z['1']['precision'])    
    r_scores.append(z['1']['recall'])
    
    print(classification_report(yt, yp))   


--------------------------------------------------------------------------------Fold # 0--------------------------------------------------------------------
Epoch 0 | Train loss: 0.300029432965499 | Validation Loss: 0.13655933489402136 
Epoch 1 | Train loss: 0.18946483818757714 | Validation Loss: 0.10540555492043495 
Epoch 2 | Train loss: 0.14054055631188958 | Validation Loss: 0.07922371911505859 
Epoch 3 | Train loss: 0.11196969468746278 | Validation Loss: 0.1049208715558052 
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       281
           1       0.99      0.94      0.96       188

    accuracy                           0.97       469
   macro avg       0.98      0.97      0.97       469
weighted avg       0.97      0.97      0.97       469

--------------------------------------------------------------------------------Fold # 1--------------------------------------------------------------------
Epoch 0 | Train loss: 0.3508546908

In [0]:
np.mean(np.array(p_scores)), np.mean(np.array(r_scores)), np.mean(np.array(f1_scores))

(0.9696186608152031, 0.9743685288428718, 0.971791202294806)

In [0]:
torch.save(auto_model.state_dict(), 'models/bert_cased_sym_attn_97_10fold.pth')

In [0]:
# auto_model.load_state_dict(torch.load('models/bert_new.pth'))
# print(f'Done')

# Analysis of model

In [0]:
y_pred = []
y_true = []

x = []

for batch in tqdm(validation_dataloader):
  
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    outputs = auto_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
    logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  prob = softmax(logits)
  pred = np.argmax(prob, axis=1)
  for i in b_input_ids:
    z = []
    for j in i:
      z.append(j.item())
    x.append(z)
  [y_pred.append(i) for i in prob[:,1]]
  [y_true.append(i) for i in label_ids]

In [0]:
df = pd.DataFrame({'Prob':y_pred, 'class':y_true, 'input':x})
df.head()

In [0]:
#tok1.decode(df.loc[1,'input'])

df['sentence'] = df['input'].apply(lambda x:tok1.decode(x))
df.head()

In [0]:
df['Correct'] = ((df['Prob']>=0.5)*1 == df['class'])*1
w = df[df['Correct']==0].copy()
w.head()

In [0]:
w.sort_values('Prob', inplace=True)
w.shape

In [0]:
fp = w[w['class']==0].reset_index(drop=True)
fn = w[w['class']==1].reset_index(drop=True)

In [0]:
fp.head()

In [0]:
for i in range(10):
  print(fp.loc[i,'sentence'])

In [0]:
for i in range(10):
  print(fn.loc[i,'sentence'])